In [1]:
import os, ssl, sys

if (not os.environ.get('PYTHONHTTPSVERIFY', '') and
getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context
sys.path.append('../code/')

import numpy as np
import tensorflow as tf
import gpflow

from pathlib import Path
from gpflow.likelihoods import Gaussian
from gpflow.kernels import SquaredExponential, White
from gpflow.optimizers import Scipy
from gpflow.utilities import print_summary, triangular
from gpflow.base import Parameter
from scipy.cluster.vq import kmeans2
from scipy.stats import norm
from scipy.special import logsumexp

from datasets import Datasets
from dgp import DGP, DGPIndependent

output_logdir = '/tmp/tensorboard'

!rm -rf '{output_logdir}'
!mkdir '{output_logdir}'

%load_ext tensorboard

def enumerated_logdir(_logdir_id: int = [0]):
    logdir = Path(output_logdir, str(_logdir_id[0]))
    _logdir_id[0] += 1
    return str(logdir)

In [2]:
datasets = Datasets(data_path='../data/')

In [3]:
data = datasets.all_datasets['boston'].get_data()
X, Y, Xs, Ys, Y_std = [data[_] for _ in ['X', 'Y', 'Xs', 'Ys', 'Y_std']]
batch_size = 1000
hidden_dim = X.shape[1]

In [4]:
Z = kmeans2(X, 50, minit='points')[0]
train_dataset = tf.data.Dataset.from_tensor_slices((X, Y)).repeat()\
    .prefetch(X.shape[0]//2)\
    .shuffle(buffer_size=(X.shape[0]//2))\
    .batch(batch_size)

In [5]:
kernels = []
dims = []
for l in range(2):
    kernels.append(SquaredExponential() + White(variance=1e-5))
    if l == 0:
        dims.append(X.shape[1])
    else:
        dims.append(hidden_dim)
dims.append(Y.shape[1])

dgp_model = DGPIndependent(X, Y, Z, dims, kernels, Gaussian(variance=0.05),
                           num_samples=1, num_data=X.shape[0])

# start inner layers deterministically
for layer in dgp_model.layers[:-1]:
    layer.q_sqrt = Parameter(layer.q_sqrt.value() * 1e-5, 
                             transform=triangular())

In [6]:
optimiser = tf.optimizers.Adam(0.01)

def optimisation_step(model):
    with tf.GradientTape() as tape:
        tape.watch(model.trainable_variables)
        obj = - model.elbo(X, Y, full_cov=False)
        grad = tape.gradient(obj, model.trainable_variables)
    optimiser.apply_gradients(zip(grad, model.trainable_variables))

In [9]:
def monitored_training_loop(model, logdir,
                            epochs=1, logging_epoch_freq=10):
    summary_writer = tf.summary.create_file_writer(logdir)
    tf_optimisation_step = tf.function(optimisation_step)
#     tf_optimisation_step = optimisation_step
    
    with summary_writer.as_default():
        for epoch in range(epochs):
            
            tf_optimisation_step(model)
                
            epoch_id = epoch + 1
            if epoch_id % logging_epoch_freq == 0:
                tf.print(f'Epoch {epoch_id}: ELBO (train) {model.elbo(X, Y)}')

In [10]:
monitored_training_loop(dgp_model, logdir=enumerated_logdir(), 
                        epochs=5000, logging_epoch_freq=200)

Epoch 200: ELBO (train) -657.0884842295068
Epoch 400: ELBO (train) -545.4612692843815
Epoch 600: ELBO (train) -453.8688829882278
Epoch 800: ELBO (train) -436.030970166777
Epoch 1000: ELBO (train) -421.4283116149536
Epoch 1200: ELBO (train) -419.4696927136734
Epoch 1400: ELBO (train) -403.44103498228384


KeyboardInterrupt: 

In [31]:
test_ll = model.log_likelihood(Xs, Ys)

In [33]:
print(test_ll / Xs.shape[0])

tf.Tensor(-3.0104994570009054, shape=(), dtype=float64)
